In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

train_path = "./data/train_preliminary/"
test_path = "./data/test/"

In [3]:
print("Loading data ... ")
user = pd.read_csv(train_path + "user.csv")
ad = pd.read_csv(train_path + "ad.csv")
log = pd.read_csv(train_path + "click_log.csv")
test_log = pd.read_csv(test_path + "click_log.csv")
print("Data loaded !")    

Loading data ... 
Data loaded !


In [29]:
print("Processing data ...")
# logs = pd.concat([log, test_log])
log['creative_id_str'] = log['creative_id'].map(lambda x: str(x))
sent = log.groupby(['user_id']).apply(lambda x: x['creative_id_str'].tolist())
sent2 = log.groupby(['user_id']).apply(lambda x: x['creative_id_str'])
print("Finished !")

Processing data ...
Finished !


In [32]:
sent

user_id
1         [71691, 821396, 209778, 877468, 3068256, 16837...
2         [4194247, 3765766, 22013, 155822, 63441, 16575...
3         [2556703, 3255755, 710859, 3386299, 825434, 21...
4         [1962706, 2741116, 574787, 2965543, 2264105, 3...
5         [2386115, 2326983, 75011, 3804062, 1384893, 43...
6         [1122285, 2851271, 1122285, 11212, 2145533, 21...
7         [648402, 2723246, 1483377, 126311, 2069959, 40...
8         [3320149, 1442706, 3514114, 2361397, 3521819, ...
9         [2171867, 2822877, 88325, 3042304, 1217767, 15...
10        [134938, 69204, 197464, 1626693, 42577, 333178...
11        [3483750, 71688, 2823222, 1232816, 35568, 1232...
12        [2422601, 1387469, 33918, 2533390, 33918, 4506...
13        [2278878, 1277386, 2574736, 1599446, 468194, 2...
14        [53070, 675360, 53070, 2915745, 83505, 2915745...
15        [494676, 4315931, 2056403, 162488, 2524263, 51...
16        [3374786, 1505826, 3466457, 2988201, 214469, 3...
17        [2409103, 2281342, 692

In [33]:
train_data = user
train_data['log'] = train_data['user_id'].map(lambda x: " ".join(sent[x]))

In [34]:
train_data.head()

,user_id,age,gender,log
0,1,4,1,71691 821396 209778 877468 3068256 1683713 194...
1,2,10,1,4194247 3765766 22013 155822 63441 1657530 625...
2,3,7,2,2556703 3255755 710859 3386299 825434 2198902 ...
3,4,5,1,1962706 2741116 574787 2965543 2264105 3321332...
4,5,4,1,2386115 2326983 75011 3804062 1384893 43235 14...


In [35]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=2000).fit(train_data['log'].iloc[:].values)
train_tfidf = tfidf.transform(train_data['log'].iloc[:].values)

skf = StratifiedKFold(n_splits=5, random_state=7) 
test_pred = np.zeros((test_tfidf.shape[0], 14), dtype=np.float32)  
for idx, (train_index, valid_index) in enumerate(skf.split(train_tfidf, train_data['log'].values)):

    x_train_, x_valid_ = train_tfidf[train_index], train_tfidf[valid_index]
    y_train_, y_valid_ = train_data['gender'].values[train_index], train_data['log'].values[valid_index]
    
    clf = LGBMClassifier()
    clf.fit(x_train_, y_train_)
    val_pred = clf.predict(x_valid_)
    
    print(f1_score(y_valid_, val_pred, average='macro'))

KeyboardInterrupt: 